**Required**
* Connect to FHIR Synthetic Patient Data
* Connect to OHDSI Synthetic Patient Data
* Connect to HUSH+ Synthetic Patient Data

**Questions**
* What are potentially diagnostic cluster groups to inform drug selection?
* Are there differences in exposures among patients diagnosed with Asthma during childhood vs during adulthood?
* Are there differences in medications prescribed off-label by patients diagnosed with Asthma during childhood vs during adulthood?

In [24]:
## All the imports we need
from urllib2 import Request, urlopen, URLError
from urllib import quote_plus

import mysql.connector

import pprint, json, requests

from datetime import datetime, timedelta
from dateutil.parser import parse as parse_date
from greentranslator.api import GreenTranslator

import dateutil

DISEASE2SYMPTOMS = [x.split("\t") for x in urlopen("https://www.nature.com/article-assets/npg/ncomms/2014/140626/ncomms5212/extref/ncomms5212-s4.txt").read().split("\n")]
DISEASE2SYMPTOMS = filter(lambda x: len(x) == 4, DISEASE2SYMPTOMS)
print DISEASE2SYMPTOMS[:5]

[['MeSH Symptom Term', 'MeSH Disease Term', 'PubMed occurrence', 'TFIDF score'], ['Aging, Premature', 'Respiratory Syncytial Virus Infections', '1', '3.4645514895379677'], ['Aging, Premature', 'Orthomyxoviridae Infections', '1', '3.4645514895379677'], ['Aging, Premature', 'HIV Infections', '3', '10.393654468613903'], ['Aging, Premature', 'Acquired Immunodeficiency Syndrome', '3', '10.393654468613903']]


In [25]:
#connect to UMLS
try:
    cnx = mysql.connector.connect(user='tadmin',
                                password='ncats_translator!',
                                database='umls',
                                host='translator.ceyknq0yekb3.us-east-1.rds.amazonaws.com')
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Something is wrong with your user name or password")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(err)
else:
    print
    #cnx.close()

In [26]:
#Define functions to use UMLS
def findICD10(name):
    cursor = cnx.cursor()
    query = ("SELECT CUI FROM umls.MRCONSO WHERE STR='"+name+"'")
    cursor.execute(query, ())
    res = "Undef"
    for code in cursor:
        if res=="Undef":
            res = code
    if res != "Undef":
        query = ("SELECT CODE FROM umls.MRCONSO WHERE SAB='ICD10' AND CUI='"+res[0]+"'")
        cursor.execute(query, ())
        icd10 = "Undef"
        for code in cursor:
            icd10 = code
        return (icd10[0])
    return ("Undef")

def findICD9(name):
    cursor = cnx.cursor()
    query = ("SELECT CUI FROM umls.MRCONSO WHERE STR='"+name+"'")
    cursor.execute(query, ())
    res = "Undef"
    for code in cursor:
        if res=="Undef":
            res = code
    if res != "Undef":
        query = ("SELECT CODE FROM umls.MRCONSO WHERE SAB='ICD9CM' AND CUI='"+res[0]+"'")
        cursor.execute(query, ())
        icd9 = "Undef"
        for code in cursor:
            icd9 = code
        return (icd9[0])
    return ("Undef")

**Question: What are potentially diagnostic cluster groups to inform drug selection?**
* Find patients diagnosed with Asthma
* Find symptoms for Asthma
* Find occurences of symptoms in Asthma patients
* Find symptom clusters among Asthma patients

In [27]:
## Given disease/condition term, get back ICD codes
def txt2icd(txt):
    url_con = "http://api.ohdsi.org/WebAPI/vocabulary/search"
    headers = {'content-type': 'application/json'}
    params = {"QUERY": "Asthma",
              "VOCABULARY_ID": ["ICD9CM"]
    }
    response = requests.post(url_con, data=json.dumps(params), headers=headers)
    data= json.loads(response.text.decode('utf-8'))
    icd9arr=[]
    for d in data:
        icd9arr.append(d["CONCEPT_CODE"])
    return icd9arr
print txt2icd('asthma')

[u'E945.7', u'493', u'493.9', u'493.90', u'493.92', u'493.91', u'493.2', u'493.20', u'493.22', u'493.21', u'493.82', u'493.0', u'493.00', u'493.02', u'493.01', u'V17.5', u'493.1', u'493.10', u'493.12', u'493.11', u'493.8', u'975.7']


In [28]:
#Find symptoms for Asthma
DISEASE2SYMPTOMS = [x.split("\t") for x in urlopen("https://www.nature.com/article-assets/npg/ncomms/2014/140626/ncomms5212/extref/ncomms5212-s4.txt").read().split("\n")]
DISEASE2SYMPTOMS = filter(lambda x: len(x) == 4, DISEASE2SYMPTOMS)
#print DISEASE2SYMPTOMS

In [29]:
# Find Asthma ICD10 codes
asthmaCodes = findICD10('Asthma')
#findICD10('Asthma')
findICD9('Fever')

u'780.60'

In [30]:
## Given disease name, get back symptoms (defined using MeSH terms) along with TFIDF scores
## Taken from https://www.nature.com/articles/ncomms5212
def disease2symptom(txt):
    s = filter(lambda x: txt.lower() in x[1].lower(), DISEASE2SYMPTOMS)
    return([(x[0], x[3]) for x in s])
disease = "Asthma"
symps = disease2symptom(disease)
print 'Found %s symptom MeSH terms for %s' % (len(symps), disease)

#for s in symps:
 #   findICD9(s)

Found 206 symptom MeSH terms for Asthma


In [31]:
## Pull JHU FHIR patients matching an ICD code
import requests
import urllib2
import json

#base_url = 'http://ictrweb.johnshopkins.edu/rest/synthetic'
#patients = base_url+"/Patient"+"?_count=50"

#req = requests.get(patients)

def findPatients(code):
    try:
        response = urllib2.urlopen("http://ictrweb.johnshopkins.edu/rest/synthetic/Condition?icd_10="+code+"")
    except:
        print "I can't find patients"
        exit()
    html = response.read()
    return (html)

In [35]:
## Pull HUSH+ patients matching a list of ICD codes
query = GreenTranslator ().get_query ()

HUSHpluspatients = query.clinical_get_patients (age='8', sex='male', 
                                        race='white', location='OUTPATIENT')
pprint.pprint (HUSHpluspatients[0])

{u'birth_date': u'2006-08-02 00:00:00',
 u'diag': {u'ICD10:B08.4': {u'2016-07-08 00:00:00': u'OUTPATIENT'},
           u'ICD10:E66.09': {u'2016-05-04 00:00:00': u'OUTPATIENT'},
           u'ICD10:F41.1': {u'2017-02-28 00:00:00': u'OUTPATIENT'},
           u'ICD10:F41.9': {u'2016-03-10 00:00:00': u'OUTPATIENT'},
           u'ICD10:F42.4': {u'2015-10-28 00:00:00': u'OUTPATIENT'},
           u'ICD10:F43.0': {u'2017-03-15 00:00:00': u'OUTPATIENT'},
           u'ICD10:F45.9': {u'2016-05-04 00:00:00': u'OUTPATIENT'},
           u'ICD10:F84.0': {u'2017-03-15 00:00:00': u'OUTPATIENT'},
           u'ICD10:F91.9': {u'2015-11-18 00:00:00': u'OUTPATIENT'},
           u'ICD10:H60.502': {u'2016-06-29 00:00:00': u'OUTPATIENT'},
           u'ICD10:H91.90': {u'2017-03-15 00:00:00': u'OUTPATIENT'},
           u'ICD10:H91.93': {u'2016-05-04 00:00:00': u'OUTPATIENT'},
           u'ICD10:J02.9': {u'2017-01-20 00:00:00': u'OUTPATIENT'},
           u'ICD10:J06.9': {u'2015-12-17 00:00:00': u'OUTPATIENT'},
   

**Question: Are there differences in exposures among patients diagnosed with Asthma during childhood vs during adulthood?**
* Find age of diagosis for Asthma patients
* Find zipcode for Asthma patients
* Find exposures by zipcode for Asthma patients
* Compare exposures of patients diagnosed with Asthma during childhood vs during adulthood

In [38]:
#Define functions needed to connect patients with exposures
def findAgeZipcode(stuff):
    res = []
    for p in stuff['entry']:
        d1 = p['assertedDate']
        id = p['subject']['reference'][8:]
        url = 'http://ictrweb.johnshopkins.edu/rest/synthetic/Patient/'+id
        txt = urlopen(url).read()
        st = json.loads(txt)
        #pprint.pprint(st)
        d2 = st['birthDate']
        zip = st['address'][0]['postalCode']
        d1 = dateutil.parser.parse(d1)
        d2 = dateutil.parser.parse(d2)
        #print d1
        #print d2
        diff = d1 - d2
        #print diff
        url = 'https://www.zipcodeapi.com/rest/lPMf5jmnBdclCZWkQwlFPJO6HkolG4N1TzgZSDnuRAPtzLOqi957STdzeBVVFIWz/multi-info.json/'+zip+'/degrees'
        txt = urlopen(url).read()
        st = json.loads(txt)
        #pprint.pprint(st)
        lat = st[zip]['lat']
        lng = st[zip]['lng']
        #print lat
        #print lng
        #print diff.days
        t = (id,d2,d1,diff.days,zip,lat,lng)
        res.append(t)
    return(res)

In [39]:
## Find age and zipcode for FHIR patients with asthma diagnoses
for c in asthmaCodes:
    
    ## Get asthma patients
    url = "http://ictrweb.johnshopkins.edu/rest/synthetic/Condition?icd_10="+c+"&_count=1000"
    txt = urlopen(url).read()
    stuff = json.loads(txt)
    pprint.pprint(findAgeZipcode(stuff)) 
    

[(u'811895497',
  datetime.datetime(1960, 4, 13, 0, 0),
  datetime.datetime(1961, 8, 24, 0, 0),
  498,
  u'20002',
  79.585041,
  -34.584875),
 (u'282146129',
  datetime.datetime(1981, 6, 28, 0, 0),
  datetime.datetime(1999, 3, 2, 0, 0),
  6456,
  u'20020',
  74.383789,
  -74.11438),
 (u'871346650',
  datetime.datetime(1988, 8, 30, 0, 0),
  datetime.datetime(1996, 9, 17, 0, 0),
  2940,
  u'20009',
  65.27519,
  -26.044371),
 (u'585518630',
  datetime.datetime(1986, 2, 18, 0, 0),
  datetime.datetime(1998, 5, 7, 0, 0),
  4461,
  u'20007',
  49.800576,
  -36.97239),
 (u'570453905',
  datetime.datetime(1973, 5, 24, 0, 0),
  datetime.datetime(2014, 12, 14, 0, 0),
  15179,
  u'20019',
  87.019856,
  -43.712093),
 (u'586131959',
  datetime.datetime(2001, 8, 16, 0, 0),
  datetime.datetime(2003, 1, 1, 0, 0),
  503,
  u'20032',
  39.979619,
  -20.85681),
 (u'907322427',
  datetime.datetime(1985, 1, 8, 0, 0),
  datetime.datetime(1998, 6, 20, 0, 0),
  4911,
  u'20011',
  63.016762,
  -52.385533),


  datetime.datetime(1982, 2, 7, 0, 0),
  10411,
  u'20745',
  92.664979,
  -48.347897),
 (u'942155677',
  datetime.datetime(1997, 8, 6, 0, 0),
  datetime.datetime(2010, 5, 4, 0, 0),
  4654,
  u'20016',
  53.260896,
  -58.182142),
 (u'288714129',
  datetime.datetime(1973, 11, 5, 0, 0),
  datetime.datetime(1984, 3, 11, 0, 0),
  3779,
  u'22202',
  46.305302,
  -39.23615),
 (u'178447726',
  datetime.datetime(1993, 5, 1, 0, 0),
  datetime.datetime(2002, 8, 9, 0, 0),
  3387,
  u'20018',
  79.606583,
  -40.878245),
 (u'587497820',
  datetime.datetime(1962, 5, 22, 0, 0),
  datetime.datetime(1967, 5, 3, 0, 0),
  1807,
  u'20032',
  80.086664,
  -52.369489),
 (u'624980695',
  datetime.datetime(1976, 10, 8, 0, 0),
  datetime.datetime(1994, 3, 11, 0, 0),
  6363,
  u'20032',
  55.449479,
  -68.412307),
 (u'350610149',
  datetime.datetime(1957, 12, 2, 0, 0),
  datetime.datetime(1985, 4, 17, 0, 0),
  9998,
  u'20815',
  74.506743,
  -33.534644),
 (u'248179855',
  datetime.datetime(1981, 11, 6, 0, 0)

  -32.252937),
 (u'512133636',
  datetime.datetime(1984, 12, 15, 0, 0),
  datetime.datetime(2002, 2, 15, 0, 0),
  6271,
  u'20002',
  93.336028,
  -39.168538),
 (u'357839688',
  datetime.datetime(1974, 11, 2, 0, 0),
  datetime.datetime(2013, 2, 15, 0, 0),
  13985,
  u'20005',
  56.093426,
  -70.728775),
 (u'300900954',
  datetime.datetime(2005, 9, 19, 0, 0),
  datetime.datetime(2011, 3, 20, 0, 0),
  2008,
  u'20743',
  45.186775,
  -67.7259),
 (u'407912784',
  datetime.datetime(1958, 8, 25, 0, 0),
  datetime.datetime(1967, 3, 14, 0, 0),
  3123,
  u'20032',
  56.022437,
  -51.223573),
 (u'284843040',
  datetime.datetime(1936, 8, 15, 0, 0),
  datetime.datetime(1976, 3, 11, 0, 0),
  14453,
  u'20748',
  70.327534,
  -48.858406),
 (u'228182163',
  datetime.datetime(1988, 11, 8, 0, 0),
  datetime.datetime(1992, 8, 16, 0, 0),
  1377,
  u'20003',
  92.739923,
  -64.386132),
 (u'490245137',
  datetime.datetime(1956, 11, 4, 0, 0),
  datetime.datetime(2014, 2, 19, 0, 0),
  20926,
  u'20743',
  7

TypeError: list indices must be integers, not str

**Question: Are there differences in medications prescribed off-label by patients diagnosed with Asthma during childhood vs during adulthood?**
* Find age of diagosis for Asthma patients
* Find medications taken by Asthma patients
* Find medication classes for medications taken by Asthma patients
* Compare medication classes taken by patients diagosed with Asthma during childhood vs during adulthood

In [40]:
## Pull meds for HUSH+ patients
meds = {}
for x in HUSHpluspatients:
    medList = x['medList']
    # Collect the unique meds
    for m in medList.keys():
        found = False
        try:
            meds[medList[m]] = meds[medList[m]]+1
        except KeyError:
            meds[medList[m]] = 1
pprint.pprint(meds)

{None: 9,
 u'0.3 ML Epinephrine 1 MG/ML Auto-Injector [Epipen]': 1,
 u'120 ACTUAT Fluticasone propionate 0.05 MG/ACTUAT Nasal Inhaler': 2,
 u'24 HR Methylphenidate Hydrochloride 10 MG Extended Release Oral Capsule [Ritalin]': 1,
 u'ACYCLOVIR 5 % TOPICAL OINTMENT': 1,
 u'ADVAIR HFA 230-21 MCG INHALER': 1,
 u'ALBUTEROL 90 MCG INHALER': 1,
 u'ALBUTEROL SULF HFA 90 MCG INH': 7,
 u'ALBUTEROL SULFATE 2.5 MG/3 ML (0.083 %) SOLUTION FOR NEBULIZATION': 9,
 u'ALBUTEROL SULFATE HFA 90 MCG/ACTUATION AEROSOL INHALER': 19,
 u'AMOXICILLIN 250 MG CHEWABLE TABLET': 1,
 u'AMOXICILLIN 875 MG TABLET': 1,
 u'AUGMENTIN 400-57 MG/5 ML': 1,
 u'AUGMENTIN ES-600 SUSPENSION': 3,
 u'AZITHROMYCIN 250 MG TABLET': 11,
 u'Acetaminophen 21.7 MG/ML / Hydrocodone Bitartrate 0.5 MG/ML Oral Solution': 1,
 u'Acetaminophen 32 MG/ML Oral Suspension': 1,
 u'Albuterol 0.83 MG/ML Inhalant Solution': 3,
 u'Amitriptyline Hydrochloride 10 MG Oral Tablet': 1,
 u'Amoxicillin 120 MG/ML / Clavulanate 8.58 MG/ML Oral Suspension': 4,
 u

In [41]:
## For a given medication string get NCBO annotations
## We let NCBO match any ontology since just using RxNORM doesn't
## always give us just the drug name (e.g., "CLINDAMYCIN 15 MG/ML ORAL SOLUTION" is
## a valid RxNORM term)
def med2rxnorm(txt):
    url = 'http://data.bioontology.org/annotator?text=%s&apikey=b792dd1b-cdc2-4cc8-aaf2-4fa4fbf47e4e'
    txt = urlopen(url % quote_plus(txt)).read()
    resp = json.loads(txt)
    if len(resp) == 0: return([])
    annos = []
    for aresp in resp:
        annos.extend([ x['text'] for x in aresp['annotations'] ])
    ##annos = filter(lambda x: not any(d in x for d in'0123456789'), annos)
    return(annos)
print med2rxnorm("CLINDAMYCIN 15 MG/ML ORAL SOLUTION")        


[u'CLINDAMYCIN 15 MG/ML ORAL SOLUTION', u'CLINDAMYCIN 15 MG/ML', u'CLINDAMYCIN', u'ORAL SOLUTION']


In [42]:
## Get RxNORM codes for medication strings
medrxnorm = {}
for med in meds.keys():
    if med is None: continue    
    annos = med2rxnorm(med)
    print 'Processing %s and found %d annotations' % (med, len(annos))
    medrxnorm[med] = {'count':meds[med], 'annos':annos}    

Processing HYDROCORTISONE VAL 0.2% CREAM and found 1 annotations
Processing CLARITIN 5 MG/5 ML SYRUP and found 1 annotations
Processing CLINDAMYCIN 150 MG/ML INJECTION SOLUTION and found 4 annotations
Processing PROAIR HFA 90 MCG INHALER and found 1 annotations
Processing ALBUTEROL SULFATE 2.5 MG/3 ML (0.083 %) SOLUTION FOR NEBULIZATION and found 2 annotations
Processing Fexofenadine hydrochloride 30 MG Oral Tablet and found 5 annotations
Processing ADVAIR HFA 230-21 MCG INHALER and found 1 annotations
Processing Ofloxacin 3 MG/ML Ophthalmic Solution and found 4 annotations
Processing levocetirizine dihydrochloride 5 MG Oral Tablet and found 5 annotations
Processing DIAZEPAM 2 MG TABLET and found 2 annotations
Processing Clonidine Hydrochloride 0.1 MG Oral Tablet and found 5 annotations
Processing LIDEX 0.05% OINTMENT and found 0 annotations
Processing KETOCONAZOLE 2 % TOPICAL CREAM and found 3 annotations
Processing BACTROBAN 2% OINTMENT and found 1 annotations
Processing FOCALIN XR 5

Processing Triamcinolone Acetonide 0.001 MG/MG Topical Ointment and found 5 annotations
Processing Glucose 50 MG/ML / Sodium Chloride 0.154 MEQ/ML Injectable Solution and found 7 annotations
Processing SUPRAX 200 MG/5 ML SUSPENSION and found 1 annotations
Processing LORTAB 7.5-500 MG/15 ML ELIXIR and found 1 annotations
Processing lisdexamfetamine dimesylate 50 MG Oral Capsule and found 5 annotations
Processing lansoprazole 30 MG Delayed Release Oral Capsule and found 5 annotations
Processing TAMIFLU 12 MG/ML SUSPENSION and found 1 annotations
Processing Amoxicillin 500 MG Oral Capsule and found 4 annotations
Processing Azithromycin 40 MG/ML Oral Suspension and found 4 annotations
Processing CONCERTA ER 27 MG TABLET and found 1 annotations
Processing Cyproheptadine hydrochloride 4 MG Oral Tablet and found 5 annotations
Processing AMOXICILLIN 250 MG CHEWABLE TABLET and found 4 annotations
Processing moxifloxacin 5 MG/ML Ophthalmic Solution and found 4 annotations
Processing AUGMENTIN ES

In [56]:
## Given a drug (identified by RXNORM), get conditions that the drug has a 
## status (approved, phase 3) for, via NDK API
def drug2conditions(drug):
    import urllib
    url = "https://tripod.nih.gov/ndk/treatment/%s/conditions" % (drug)
    page = urllib.urlopen(url).read().strip()
    if page == "": return []
    print page
    try:
        resp = json.loads(page)
    handle ValueError:
        return []
    conds = []
    for aresp in resp:
        condname = aresp['name']
        if 'ICD10' in aresp.keys():
            condicd10 = aresp['ICD10']
        else: condicd10 = []
        conds.append( (aresp['status'], condname, condicd10) )
    return(conds)

SyntaxError: invalid syntax (<ipython-input-56-22b2f86d4cca>, line 11)

In [53]:
keys = list(medrxnorm.keys())
n = 10
annos = medrxnorm[keys[n]]['annos']
conds = []
print annos
for anno in annos:
    conds.extend(drug2conditions(anno))
pprint.pprint(keys[n])
pprint.pprint(conds)
#"##", keys[n], "##", conds

[u'SALICYLIC ACID 275 MG/ML TOPICAL SOLUTION', u'SALICYLIC ACID 275 MG/ML', u'SALICYLIC ACID', u'TOPICAL SOLUTION']
u'Salicylic Acid 275 MG/ML Topical Solution'
[(u'Approved', u'Psoriasis', [u'L40']),
 (u'Approved', u'Keratoderma blennorrhagicum', []),
 (u'Approved', u'Pityriasis rubra pilaris', [u'L44.0']),
 (u'Approved', u'Folliculitis ulerythematosa reticulata', [u'L66.4']),
 (u'Approved', u'Congenital ichthyosis', [u'Q80']),
 (u'Approved', u'Wart', []),
 (u'Approved', u'Corns and callosities', [u'L84']),
 (u'Approved', u'Acne', [u'L70']),
 (u'Approved', u'Pyoderma', [u'L08.0']),
 (u'Approved', u'Inflammation', [])]


In [57]:
for p in HUSHpluspatients:
    diags = []
    for d in p['diag']:
        if d[0:5]=="ICD9:":
            diags.append(d)
    for m in p['medList'].keys():
        med = p['medList'][m]
    print med
    rxnorm = med2rxnorm(med)
    annos = medrxnorm[med]['annos']
    pprint.pprint(annos)
    conds = []
    for a in annos:
        print a
        x = drug2conditions(a)
        print x
        conds.extend(drug2conditions(a))
    print conds
    break
        

PERMETHRIN 5 % TOPICAL CREAM
[u'PERMETHRIN 5 % TOPICAL CREAM', u'PERMETHRIN', u'TOPICAL CREAM']
PERMETHRIN 5 % TOPICAL CREAM
<html>
<head><title>400 Bad Request</title></head>
<body bgcolor="white">
<center><h1>400 Bad Request</h1></center>
<hr><center>nginx</center>
</body>
</html>


ValueError: No JSON object could be decoded